<a href="https://colab.research.google.com/github/EMAT31530/ai-group-project-group-2/blob/master/WordleRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Packages

In [ ]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install stable_baselines3

     |████████████████████████████████| 320.4 MB 53 kB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 459 kB 45.4 MB/s 
     |████████████████████████████████| 2.9 MB 43.7 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
 

     |████████████████████████████████| 52 kB 586 kB/s 
     |████████████████████████████████| 176 kB 4.4 MB/s 


In [ ]:
#Environment
import gym
import pandas as pd
from gym import Env
from stable_baselines3.common.env_util import make_vec_env

#Import Worlde_gym
from google.colab import drive
drive.mount('/content/drive')


#DL packages
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

#RL packages
from rl.agents import DQNAgent #not sure if this is the right agent
#from rl.policy import EpsilonGreedyPolicy # Corrent policy but unable to install as cant find the name
from rl.memory import SequentialMemory
from rl.core import Agent

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New section

Environment

In [ ]:
from gym.envs.registration import register
#register(id="wordle-v0", entry_point="/content/drive/MyDrive/wordle_gym/envs/wordle_env.py:WordleEnv",)
#guess= pd.read_csv('/content/drive/MyDrive/wordle_gym/words/guess.csv')
#solution = pd.read_csv('/content/drive/MyDrive/wordle_gym/words/solution.csv')



In [14]:
import sys
sys.path.append('/content/drive/MyDrive/wordle_gym')

In [16]:
import gym
import sys

from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env


import wordle_gym
sys.path.append("..")
sys.path.append("../..")


# Parallel environments
def train_model():    
    


    env = make_vec_env("wordle-v0", n_envs=10)

    model = DQN(
        "MlpPolicy", 
        env,
        gamma=0.99, 
        learning_rate=5e-4,
        learning_starts=100000,
        buffer_size=10000,
        exploration_fraction=1,
        exploration_final_eps=0.5,
        target_update_interval=1000,
        train_freq=4,
        verbose=1,
    )
    try:
        model.learn(total_timesteps=1e7, log_interval=1000)
    except KeyboardInterrupt:
        pass
    model.save("wordle_dqn")
    return model

train_model()

model = DQN.load("wordle_dqn")

env = gym.make("wordle-v0")
obs = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    print("Word:", env._valid_words[action][0])
    obs, rewards, done, info = env.step(action)
    env.render()

Using cpu device
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6        |
|    ep_rew_mean      | -7.2     |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 1000     |
|    fps              | 18116    |
|    time_elapsed     | 0        |
|    total_timesteps  | 6010     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6        |
|    ep_rew_mean      | -7.4     |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes         | 2000     |
|    fps              | 19576    |
|    time_elapsed     | 0        |
|    total_timesteps  | 12020    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6.01     |
|    ep_rew_mean      | -7.43    |
|    exploration_rate | 0.999    |
| time/               |          |
|  

Building Model and RL agent

In [ ]:
#Building DL model
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

#building RL agent
def build_agent(model, actions):
  #input epsilon greedy policy
    #policy = 
    memory = SequentialMemory(limit=10000, window_length=1)
    agent = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=5, target_model_update=1e-2)
    return agent

In [ ]:
model = build_model(states, actions)
model.summary()

RL agent

In [ ]:
agent = build_agent(model, actions)
agent.compile(Adam(lr=1e-3), metrics=['mae'])
agent.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Need to load environment from memory and apply RL agent to it